# Tile netCDF and write out geotiff

In [1]:
%pylab notebook

from datacube.helpers import write_geotiff
from datacube.utils.geometry import CRS
import glob
import xarray as xr

Populating the interactive namespace from numpy and matplotlib


## Read in the netCDF files created with datacube-stats

### Point to the folder where datacube-stats wrote out the tiles

In [2]:
statsoutputs = '/g/data/r78/cek156/datacube_stats/Border_WOFS/'

### Find all the tiles

In [3]:
Alltiles = glob.glob('{0}*.nc'.format(statsoutputs))
Alltiles

['/g/data/r78/cek156/datacube_stats/Border_WOFS/wofs_summary_18_-34_19990801.nc',
 '/g/data/r78/cek156/datacube_stats/Border_WOFS/wofs_summary_18_-35_19990801.nc',
 '/g/data/r78/cek156/datacube_stats/Border_WOFS/wofs_summary_19_-34_19990801.nc',
 '/g/data/r78/cek156/datacube_stats/Border_WOFS/wofs_summary_19_-33_19990801.nc',
 '/g/data/r78/cek156/datacube_stats/Border_WOFS/wofs_summary_17_-34_19990801.nc',
 '/g/data/r78/cek156/datacube_stats/Border_WOFS/wofs_summary_17_-33_19990801.nc',
 '/g/data/r78/cek156/datacube_stats/Border_WOFS/wofs_summary_16_-34_19990801.nc',
 '/g/data/r78/cek156/datacube_stats/Border_WOFS/wofs_summary_18_-33_19990801.nc',
 '/g/data/r78/cek156/datacube_stats/Border_WOFS/wofs_summary_16_-33_19990801.nc']

### Iteratively open all of the files, and combine them into a single xarray

In [4]:
for ix, tiles in enumerate(Alltiles):
    outputname = '{0}tif'.format(tiles[:-2])
    !gdal_translate -of GTiff NETCDF:$tiles:frequency $outputname
    Tile = xr.open_rasterio(outputname)
    if ix == 0:
        CombinedWOFS = Tile
    else:
        CombinedWOFS = CombinedWOFS.combine_first(Tile)

Input file size is 4000, 4000
0...10...20...30...40...50...60...70...80...90...100 - done.
Input file size is 4000, 4000
0...10...20...30...40...50...60...70...80...90...100 - done.
Input file size is 4000, 4000
0...10...20...30...40...50...60...70...80...90...100 - done.
Input file size is 4000, 4000
0...10...20...30...40...50...60...70...80...90...100 - done.
Input file size is 4000, 4000
0...10...20...30...40...50...60...70...80...90...100 - done.
Input file size is 4000, 4000
0...10...20...30...40...50...60...70...80...90...100 - done.
Input file size is 4000, 4000
0...10...20...30...40...50...60...70...80...90...100 - done.
Input file size is 4000, 4000
0...10...20...30...40...50...60...70...80...90...100 - done.
Input file size is 4000, 4000
0...10...20...30...40...50...60...70...80...90...100 - done.


In [5]:
CombinedWOFS = CombinedWOFS.where(CombinedWOFS != -1)

## Write out our combined array wofs to GeoTiff
(this way we can just read this file in straight away in the future and don't need to keep combining the tiles)

In [6]:
wofsds = CombinedWOFS.to_dataset(name = 'frequency')
wofsds.attrs['transform'] = CombinedWOFS.transform
wofsds.attrs['crs'] = CRS(CombinedWOFS.crs)

write_geotiff('{0}Combined.tif'.format(statsoutputs), wofsds.squeeze())